In [4]:
import os
print(os.getcwd())
os.chdir("../")
os.getcwd()

/home/mobo/Documents/Ineuron/Live_Class/DL/DL_live_notes/CNN/deepCNNClassifier/research


'/home/mobo/Documents/Ineuron/Live_Class/DL/DL_live_notes/CNN/deepCNNClassifier'

In [1]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [2]:
from deepClassifier.constants import (CONFIG_FILE_PATH,
                                      PARAMS_FILE_PATH)
from deepClassifier.utils import create_directories, read_yaml


class ConfigurationManager:
    
    def __init__(self,
                 config_filepath: Path=CONFIG_FILE_PATH,
                 params_filepath: Path=PARAMS_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_prepare_callbacks_config(self) -> PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        
        create_directories([config.root_dir,
                            config.tensorboard_root_log_dir,
                            config.checkpoint_model_filepath])
        
        return PrepareCallbackConfig(
            root_dir=config.root_dir,
            tensorboard_root_log_dir=config.tensorboard_root_log_dir,
            checkpoint_model_filepath=config.checkpoint_model_filepath
        )

In [32]:
import tensorflow as tf
from typing import List
import time


class PrepareCallbacks:
    
    def __init__(self, 
                 prepare_callback_config: PrepareCallbackConfig):
        self.config = prepare_callback_config
    
    @property
    def _prepare_tb_callback(self) -> tf.keras.callbacks.TensorBoard:
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tensorboard_root_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        
        return tf.keras.callbacks.TensorBoard(
            log_dir=tensorboard_root_log_dir
        )
    
    @property
    def _prepare_ckpt_callback(self) -> tf.keras.callbacks.ModelCheckpoint:
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
        
    def get_tb_ckpt_callbacks(self) -> List:
        return [
            self._prepare_tb_callback,
            self._prepare_ckpt_callback
            ]
        

In [33]:
config = ConfigurationManager()
prepare_callback_config=config.get_prepare_callbacks_config()
callbacks = PrepareCallbacks(
    prepare_callback_config=prepare_callback_config
)
callbacks.get_tb_ckpt_callbacks()

[2022-09-22 12:46:12,132: INFO: common]: yaml file: configs/config.yaml loaded successfully
[2022-09-22 12:46:12,136: INFO: common]: yaml file: params.yaml loaded successfully
[2022-09-22 12:46:12,138: INFO: common]: created directory at: artifacts
[2022-09-22 12:46:12,140: INFO: common]: created directory at: artifacts/prepare_callbacks
[2022-09-22 12:46:12,143: INFO: common]: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir
[2022-09-22 12:46:12,144: INFO: common]: created directory at: artifacts/prepare_callbacks/checkpoint_model_filepath
